In [65]:
import numpy as np
import math

class consts:
    k = 1.38 * 10**-23
    hbar = 1.05 * 10**-34 
    T = 300
    m = 9.1 * 10**-31
    a0_2 = 2.8 * 10**-21 # for cross section, see NIST
    e = 1.6 * 10**-19
    torr = 133
    wavelen = 1064 * 10**-9
    omega = 2 * np.pi * 3*10**8 / wavelen
    N_e = 10**13

    t1 = 40 * 10**-9
    r = 18e-6

In [66]:
def needed_probability( N_a):
    return consts.N_e / (4/3*np.pi * consts.r**3 * N_a * consts.t1)

def field_multiphoton( ionisation, probability):
    n = math.ceil(ionisation / (consts.omega * consts.hbar))
    return consts.omega / consts.e * np.sqrt( 8 * consts.m * ionisation / np.e) * (probability / (consts.omega * n**1.5))**(0.5/n)

def nu( energy, cross):
    return 1 / (consts.k * consts.T) * np.sqrt(2 * energy / consts.m) * cross * consts.torr

def field( ionisation, nu_m):
    return np.sqrt( consts.m * consts.omega**2 * ionisation / (consts.e**2 * nu_m * consts.t1) * 43)

def avalanche( cross_section,      # m^2
               electron_energy,    # eV
               excitation_energy,  # eV
               pressure):          # torr
    print( "-------Avalanche-------")
    print( "cross_section = {:.3g} m^2".format( cross_section))
    print( "electron_energy = {:.3g} eV".format( electron_energy))
    print( "excitation_energy = {:.3g} eV".format( excitation_energy))
    print( "pressure = {:.3g} torr".format( pressure))

    nu_m_torr = nu( electron_energy * consts.e, cross_section) # Hz/torr
    print( "nu_m = {:.3g}*pressure = {:.3g} Hz".format( nu_m_torr, nu_m_torr * pressure))
    E = field( excitation_energy * consts.e, nu_m_torr * pressure) # V/m
    S = (E/100 / 19)**2 # W/cm^2
    print( "E = {:.3g} V/cm".format( E / 100))
    print( "S = {:.3g} W/cm^2".format( S))
    print( "")

def real( E = None, # V/cm
          S = None): # W/cm
    assert (E and not S) or (S and not E), "Specify only one parameter"

    print ("---------Real---------")
    if (E and not S):
        S = (E / 19)**2
    else:
        E = 19 * S**0.5

    print( "E = {:.3g} V/cm".format( E))
    print( "S = {:.3g} W/cm^2".format( S))
    print( "")

def multiphoton( concentration, # 1/m^3
                 ionisation_energy): # eV
    print( "-------Multiphoton-------")
    print( "concentration = {:.3g} 1/m^3".format( concentration))
    print( "ionisation_energy = {:.3g} eV".format( ionisation_energy))
    probability = needed_probability( concentration)
    print( "probability = {:.3g} 1/s".format( probability))
    E = field_multiphoton( ionisation_energy * consts.e, probability) # V/m
    S = (E/100 / 19)**2
    print( "E = {:.3g} V/cm".format( E / 100))
    print( "S = {:.3g} W/cm^2".format( S))
    print( "")


In [67]:
# Just to compare with Raizer nu_m

print( "Helium")

cross = 4.2 * consts.a0_2
energy = 50
print( "He: W = {:.2f}, cross = {:.3g}, nu = {:.3g}*p_torr".format( energy, cross, nu(energy * consts.e, cross) ))

cross = 15 * consts.a0_2
energy = 10
print( "He: W = {:.2f}, cross = {:.3g}, nu = {:.3g}*p_torr".format( energy, cross, nu(energy * consts.e, cross) ))
print( "Raizer: nu = 2.4e+09*p_torr")

Helium
He: W = 50.00, cross = 1.18e-20, nu = 1.58e+09*p_torr
He: W = 10.00, cross = 4.2e-20, nu = 2.53e+09*p_torr
Raizer: nu = 2.4e+09*p_torr


In [88]:
print( "Xenon")

pressure = 760 # torr
N_a = pressure * consts.torr / (consts.k * consts.T)

multiphoton( concentration = N_a,
             ionisation_energy = 12.13)

#Differential cross sections for elastic electron scattering in xenon in the energy range from 5eV to 10eV
avalanche( cross_section = 30 * 10**-20,
           electron_energy = 4,
           excitation_energy = 10,
           pressure = pressure)

# Raizer, page 58, S(760 torr)
real( S = 0.6e10)


# NIST
cross = 20.5 * consts.a0_2
energy = 50
# print( "Xe: W = {:.2f}, cross = {:.3g}, nu = {:.3g}*p_torr".format( energy, cross, nu(energy * consts.e, cross) ))

Xenon
-------Multiphoton-------
concentration = 2.44e+25 1/m^3
ionisation_energy = 12.1 eV
probability = 4.19e+08 1/s
E = 1.07e+08 V/cm
S = 3.18e+13 W/cm^2

-------Avalanche-------
cross_section = 3e-19 m^2
electron_energy = 4 eV
excitation_energy = 10 eV
pressure = 760 torr
nu_m = 1.14e+10*pressure = 8.69e+12 Hz
E = 1.49e+06 V/cm
S = 6.12e+09 W/cm^2

---------Real---------
E = 1.47e+06 V/cm
S = 6e+09 W/cm^2



In [91]:
print( "Argon")

pressure = 760 # torr
N_a = pressure * consts.torr / consts.k / 300

multiphoton( concentration = N_a,
             ionisation_energy = 15.8)

# Some article
avalanche( cross_section = 10 * 10**-20,
           electron_energy = 5,
           excitation_energy = 12.5,
           pressure = pressure)

# Raizer, page 58, S(760 torr)
real( S = 2e10)

# NIST
cross = 14.5 * consts.a0_2
energy = 50
nu_m = nu(energy * consts.e, cross)
#print( "Ar: W = {:.2f}, cross = {:.3g}, nu = {:.3g}*p_torr".format( energy, cross, nu_m ))
#print( "E = {:.3g}".format( field( 15.76 * consts.e, nu_m * 760)))



Argon
-------Multiphoton-------
concentration = 2.44e+25 1/m^3
ionisation_energy = 15.8 eV
probability = 4.19e+08 1/s
E = 1.45e+08 V/cm
S = 5.83e+13 W/cm^2

-------Avalanche-------
cross_section = 1e-19 m^2
electron_energy = 5 eV
excitation_energy = 12.5 eV
pressure = 760 torr
nu_m = 4.26e+09*pressure = 3.24e+12 Hz
E = 2.72e+06 V/cm
S = 2.05e+10 W/cm^2

---------Real---------
E = 2.69e+06 V/cm
S = 2e+10 W/cm^2



In [93]:
print( "Air")

pressure = 760 # torr
N_a = pressure * consts.torr / consts.k / 300

multiphoton( concentration = N_a,
             ionisation_energy = 14)

# Some article
avalanche( cross_section = 50 * consts.a0_2,
           electron_energy = 4,
           excitation_energy = 12,
           pressure = pressure)

# Raizer, page 58, S(760 torr)
real( S = 5e11)


Air
-------Multiphoton-------
concentration = 2.44e+25 1/m^3
ionisation_energy = 14 eV
probability = 4.19e+08 1/s
E = 1.3e+08 V/cm
S = 4.69e+13 W/cm^2

-------Avalanche-------
cross_section = 1.4e-19 m^2
electron_energy = 4 eV
excitation_energy = 12 eV
pressure = 760 torr
nu_m = 5.33e+09*pressure = 4.05e+12 Hz
E = 2.38e+06 V/cm
S = 1.57e+10 W/cm^2

---------Real---------
E = 1.34e+07 V/cm
S = 5e+11 W/cm^2

